# **iMPORTING eNVIORNEMENT AND lIBRARIES**

In [ ]:
!pip install seaborn
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import os , json, re, pickle,torch
from scipy.stats import zscore,skew,kurtosis
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("/kaggle/input/twelevc/owid-co2-data.csv")
data.head()

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df.tail()

In [ ]:
print("Basic Info:")
print(df.info())

print("Summary :")
print(df.describe())



In [ ]:

print("\nShape of the DataFrame", df.shape)
print("\n The coloumns in the dataframe are :", df.columns)


# **Missing Coloumns and Imputation**

In [ ]:
print(df.isnull().sum().sort_values(ascending=False))

In [ ]:
# Compute missing %
missing_percent = df.isnull().sum() / len(df) * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
print(f"\n {len(missing_percent)} columns have missing values.")

In [ ]:
 #Chunked Missing Value Visualization – 25 at a time
import math

chunk_size = 25
missing_percent_sorted = missing_percent.sort_values(ascending=False)
num_chunks = math.ceil(len(missing_percent_sorted) / chunk_size)

for i in range(num_chunks):
    start = i * chunk_size
    end = start + chunk_size
    chunk = missing_percent_sorted[start:end]
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x=chunk.values, y=chunk.index, palette="Reds_r")
    plt.xlabel("Missing Percentage (%)")
    plt.title(f"Missing Data (Features {start+1} to {min(end, len(missing_percent_sorted))})")
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:

# Define selected core modeling features
selected_features = [
    'gdp', 'population', 'energy_per_capita', 'primary_energy_consumption',
    'energy_per_gdp', 'co2_per_unit_energy', 'cement_co2',
    'oil_co2', 'coal_co2', 'gas_co2', 'flaring_co2', 'co2_growth_prct', 'co2_per_gdp',
    'co2_per_capita', 'cumulative_co2', 'methane', 'nitrous_oxide',
    'total_ghg_excluding_lucf', 'land_use_change_co2'
]

# Define critical context columns to always keep
context_columns = ['country', 'year', 'iso_code']

# Define optional features used for dashboard/simulation (SaaS, not ML)
optional_features = [
    'co2_growth_abs', 'share_of_temperature_change_from_ghg',
    'temperature_change_from_ch4', 'temperature_change_from_co2',
    'temperature_change_from_ghg', 'temperature_change_from_n2o',
    'total_ghg'
]
target_column = ['co2']  # or whatever your target variable is

columns_to_keep = selected_features + context_columns + optional_features + target_column

# Drop all other columns (>85% null OR irrelevant)
columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

print(columns_to_drop)


In [ ]:
# Drop them
df_cleaned = df.drop(columns=columns_to_drop)

print(f"Dropped {len(columns_to_drop)} columns.")
print(f"Final dataset shape: {df_cleaned.shape}")

In [ ]:
df_cleaned.nunique()


In [ ]:
df_cleaned.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import skew, kurtosis

selected_features = [
    'gdp', 'population', 'energy_per_capita', 'primary_energy_consumption',
    'energy_per_gdp', 'co2_per_unit_energy', 'cement_co2',
    'oil_co2', 'coal_co2', 'gas_co2', 'flaring_co2', 'co2_growth_prct', 'co2_per_gdp',
    'co2_per_capita', 'cumulative_co2', 'methane', 'nitrous_oxide',
    'total_ghg_excluding_lucf', 'land_use_change_co2'
]
for col in selected_features:
    if df_cleaned[col].isnull().mean() < 0.9:  # only plot if not too sparse
        plt.figure(figsize=(6, 4))
        sns.histplot(df_cleaned[col].dropna(), kde=True)
        plt.title(f"{col} | Skewness: {skew(df_cleaned[col].dropna()):.2f}, Kurtosis: {kurtosis(df_cleaned[col].dropna()):.2f}")
        plt.axvline(df_cleaned[col].median(), color='red', linestyle='--', label='Median')
        plt.axvline(df_cleaned[col].mean(), color='blue', linestyle='--', label='Mean')
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
optional_features = [
    'co2_growth_abs', 'share_of_temperature_change_from_ghg',
    'temperature_change_from_ch4', 'temperature_change_from_co2',
    'temperature_change_from_ghg', 'temperature_change_from_n2o',
    'total_ghg'
]

for col in optional_features:
    print(f"\n{col}")
    print(f"Unique values: {df_cleaned[col].nunique()}")
    print(f"Mode: {df_cleaned[col].mode()[0]}")
    print(df_cleaned[col].value_counts(dropna=True).head())
    df_cleaned[col].hist(bins=15)
    plt.axvline(df[col].mode()[0], color='orange', linestyle='--', label='Mode')
    plt.title(f"{col} Distribution")
    plt.legend()
    plt.show()


In [ ]:

# Context columns
df_cleaned['iso_code'] = df_cleaned.groupby('country')['iso_code'].ffill()
df_cleaned['population'] = (
    df_cleaned.groupby('country')['population']
    .transform(lambda x: x.interpolate(method='linear'))
    .ffill()
    .bfill()
)


# Core modeling features – use median()
core_features_median = [
    'gdp', 'energy_per_capita', 'primary_energy_consumption',
    'energy_per_gdp', 'renewables_share_energy', 'co2_per_unit_energy',
    'cement_co2', 'oil_co2', 'coal_co2', 'gas_co2', 'flaring_co2',
    'co2_growth_prct', 'co2_per_gdp', 'co2_per_capita',
    'cumulative_co2', 'methane', 'nitrous_oxide',
    'total_ghg_excluding_lucf', 'land_use_change_co2'
]

for col in core_features_median:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].fillna(df[col].median())

# Optional/dashboard features – use mode()
optional_features_mode = [
    'co2_growth_abs',  # strong mode = 0.0
    'share_of_temperature_change_from_ghg',
    'temperature_change_from_ch4',
    'temperature_change_from_co2',
    'temperature_change_from_ghg',
    'temperature_change_from_n2o'
]

for col in optional_features_mode:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode()[0])

# `total_ghg` – use median due to weak mode dominance
if 'total_ghg' in df_cleaned.columns:
    df_cleaned['total_ghg'] = df_cleaned['total_ghg'].fillna(df_cleaned['total_ghg'].median())


In [ ]:
df_cleaned.head()

In [ ]:
from scipy.stats import skew, kurtosis

# Combine all features that were imputed
check_columns = core_features_median + optional_features_mode + ['total_ghg']

for col in check_columns:
    if col in df_cleaned.columns:
        col_data = df_cleaned[col].dropna()
        sk = skew(col_data)
        kt = kurtosis(col_data)
        print(f"{col}: Skewness = {sk:.2f}, Kurtosis = {kt:.2f}")


# **Year Corretcion**

In [ ]:
df_cleaned["year"]=pd.to_numeric(df_cleaned['year'], errors="coerce")
df_cleaned = df_cleaned[(df_cleaned["year"]>=1990) & (df_cleaned["year"]<=2024)].copy()
df_cleaned = df_cleaned.dropna(subset=["year"])
print("Cleaned Dataset. Shape:", df_cleaned.shape)
df_cleaned.head()

In [ ]:
print("\nNo. of duplicated values  is:", df_cleaned.duplicated().sum())

# del df1  # optional, clears old memory
del df1  # optional, clears old memory
del df1  # optional, clears old memory
**Evaluation of Outlier Removal**

In [ ]:

print("MEAN V/s MEDIAN  V/s MODE +  SKEW/KURTOSIS")
for col in selected_features:
    print(f"\nColumn : {col}")
    print(f"Mean:{df_cleaned[col].mean():.2f}")
    print(f"Median:{df_cleaned[col].median():.2f}")
    print(f"Mode:{df_cleaned[col].mode()[0]:.2f}")
    print(f"Skewness:{skew(df_cleaned[col].dropna()):.2f}")
    print(f"Kurotis:{kurtosis(df_cleaned[col].dropna()):.2f}")
    plt.figure(figsize=(8, 4))
    plt.axvline(df_cleaned[col].mean(), color='red', linestyle='--', label='Mean')
    plt.axvline(df_cleaned[col].median(), color='green', linestyle='--', label='Median')
    plt.axvline(df_cleaned[col].mode()[0], color='blue', linestyle='--', label='Mode')
    plt.title(f"Distribution of {col} with Central Tendencies")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# List of core modeling features (excluding 'co2')
selected_features = [
    'gdp', 'population', 'energy_per_capita', 'primary_energy_consumption',
    'energy_per_gdp', 'co2_per_unit_energy', 'cement_co2',
    'oil_co2', 'coal_co2', 'gas_co2', 'flaring_co2', 'co2_growth_prct', 'co2_per_gdp',
    'co2_per_capita', 'cumulative_co2', 'methane', 'nitrous_oxide',
    'total_ghg_excluding_lucf', 'land_use_change_co2'
]

# Visualize each feature with a horizontal boxplot
for col in selected_features:
    if col in df_cleaned.columns:
        plt.figure(figsize=(14, 6))
        sns.boxplot(data=df_cleaned, x=col, color='skyblue', linewidth=1)
        plt.title(f'Boxplot of {col}', fontsize=11)
        plt.grid(axis='x', linestyle='--', alpha=0.4)
        plt.tight_layout()
        plt.show()


In [ ]:


def evaluate_outlier_methods(df_cleaned, col):
    print(f"\n Evaluating outliers in: {col}")
    
    original = df_cleaned[col].dropna()
    original_skew = skew(original)
    original_kurt = kurtosis(original)

    # --- Method 1: Z-Score Trimming ---
    z_scores = np.abs(zscore(original))
    df_z = df_cleaned[z_scores < 3].copy()
    z_skew = skew(df_z[col].dropna())
    z_kurt = kurtosis(df_z[col].dropna())

    # --- Method 2: Percentile Capping (1st–99th) ---
    lower, upper = original.quantile([0.01, 0.99])
    df_p = df_cleaned.copy()
    df_p[col] = np.clip(df_cleaned[col], lower, upper)
    p_skew = skew(df_p[col].dropna())
    p_kurt = kurtosis(df_p[col].dropna())

    # --- Method 3: IQR Trimming ---
    Q1, Q3 = original.quantile(0.25), original.quantile(0.75)
    IQR = Q3 - Q1
    lower_iqr = Q1 - 1.5 * IQR
    upper_iqr = Q3 + 1.5 * IQR
    df_iqr = df_cleaned[(df_cleaned[col] >= lower_iqr) & (df_cleaned[col] <= upper_iqr)].copy()
    iqr_skew = skew(df_iqr[col].dropna())
    iqr_kurt = kurtosis(df_iqr[col].dropna())

    # --- Summary Table ---
    summary = pd.DataFrame({
        'Method': ['Original', 'Z-Score', 'Percentile', 'IQR'],
        'Count': [len(original), len(df_z), len(df_p), len(df_iqr)],
        'Skewness': [original_skew, z_skew, p_skew, iqr_skew],
        'Kurtosis': [original_kurt, z_kurt, p_kurt, iqr_kurt]
    })

    print(summary)

    # --- Boxplot Visualization ---
    plt.figure(figsize=(12, 4))
    sns.boxplot(data=[
        np.log1p(original),
        np.log1p(df_z[col].dropna()),
        np.log1p(df_p[col].dropna()),
        np.log1p(df_iqr[col].dropna())
    ])
    plt.xticks(ticks=[0, 1, 2, 3], labels=['Original', 'Z-Score', 'Percentile', 'IQR'])
    plt.title(f' Log-Boxplot of {col} – Z-Score, Percentile, IQR')
    plt.grid(axis='y', linestyle='--', alpha=0.4)
    plt.show()

    return summary

selected_features = [
    'gdp', 'population', 'energy_per_capita', 'primary_energy_consumption',
    'energy_per_gdp', 'co2_per_unit_energy', 'cement_co2',
    'oil_co2', 'coal_co2', 'gas_co2', 'flaring_co2', 'co2_growth_prct', 'co2_per_gdp',
    'co2_per_capita', 'cumulative_co2', 'methane', 'nitrous_oxide',
    'total_ghg_excluding_lucf', 'land_use_change_co2'
]


# Loop through and evaluate each
for col in selected_features:
    evaluate_outlier_methods(df_cleaned, col)




In [ ]:
best_outlier_methods = {
    'gdp': 'percentile',
    'population': 'iqr',
    'energy_per_capita': 'iqr',
    'primary_energy_consumption': 'iqr',
    'energy_per_gdp': 'iqr',
    'co2_per_unit_energy': 'iqr',
    'cement_co2': 'iqr',
    'oil_co2': 'percentile',
    'coal_co2': 'percentile',
    'gas_co2': 'percentile',
    'flaring_co2': 'iqr',
    'co2_growth_prct': 'iqr',
    'co2_per_gdp': 'iqr',
    'co2_per_capita': 'iqr',
    'cumulative_co2': 'percentile',
    'methane': 'iqr',
    'nitrous_oxide': 'iqr',
    'total_ghg_excluding_lucf': 'iqr',
    'land_use_change_co2': 'iqr'
}



In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore

def apply_columnwise_outlier_cleaning(df, method_map):
    df1 = df.copy()

    for col, method in method_map.items():
        original = df1[col].dropna()

        if method == 'zscore':
            z_scores = np.abs(zscore(original))
            df1.loc[z_scores >= 3, col] = np.nan  # Mark outliers as NaN (optional) or replace

        elif method == 'percentile':
            lower, upper = original.quantile([0.01, 0.99])
            df1[col] = np.clip(df1[col], lower, upper)

        elif method == 'iqr':
            Q1 = original.quantile(0.25)
            Q3 = original.quantile(0.75)
            IQR = Q3 - Q1
            lower_iqr = Q1 - 1.5 * IQR
            upper_iqr = Q3 + 1.5 * IQR
            df1.loc[(df1[col] < lower_iqr) | (df1[col] > upper_iqr), col] = np.nan

        else:
            raise ValueError(f"Unknown method: {method} for column: {col}")

    return df1

df1 = apply_columnwise_outlier_cleaning(f_cleaned, best_outlier_methods)
print("After cleaning (column-wise):", df1.shape)






In [ ]:
print("Original shape:", df_cleaned.shape)
print("After outlier removal:", df1.shape)
print("Total rows removed:", df_cleaned.shape[0] - df1.shape[0])
print("Percent data removed:", round(100 * (df_cleaned.shape[0] - df1.shape[0]) / df_cleaned.shape[0], 2), "%")


In [ ]:
df1.to_csv("twelvec_phase1_outliers_cleaned.csv", index=False)
print("\n✅ Final cleaned file saved: twelvec_phase1_outliers_cleaned.csv")


In [ ]:
print("MEAN vs MEDIAN vs MODE + SKEW/KURTOSIS (Post Outlier Removal)")

for col in selected_features:
    if col in df1.columns:
        print(f"\nColumn : {col}")
        print(f"Mean    : {df1[col].mean():.2f}")
        print(f"Median  : {df1[col].median():.2f}")
        print(f"Mode    : {df1[col].mode()[0]:.2f}")
        print(f"Skewness: {skew(df1[col].dropna()):.2f}")
        print(f"Kurtosis: {kurtosis(df1[col].dropna()):.2f}")
        
        plt.figure(figsize=(8, 4))
        sns.histplot(df1[col], kde=True, bins=50, color='skyblue')
        plt.axvline(df1[col].mean(), color='red', linestyle='--', label='Mean')
        plt.axvline(df1[col].median(), color='green', linestyle='--', label='Median')
        plt.axvline(df1[col].mode()[0], color='blue', linestyle='--', label='Mode')
        plt.title(f"Distribution of {col} with Central Tendencies (Final Cleaned)", fontsize=11)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
# Visualize each feature with a horizontal boxplot

for col in selected_features:
    if col in df1.columns:
        plt.figure(figsize=(14, 6))
        sns.boxplot(data=df1, x=col, color='skyblue', linewidth=1)
        plt.title(f'Boxplot of {col}', fontsize=11)
        plt.grid(axis='x', linestyle='--', alpha=0.4)
        plt.tight_layout()
        plt.show()



In [ ]:
from scipy.stats import skew, kurtosis
from numpy import log1p

log_candidates = []

for col in selected_features:
    if col in df1.columns:
        values = df1[col].dropna()
        if values.min() >= 0:  # log1p safe
            sk = skew(values)
            kt = kurtosis(values)
            if sk > 2 and kt > 7:
                log_candidates.append(col)

print("📌 Features safe for log1p():", log_candidates)

# Apply log1p
for col in log_candidates:
    df2[col] = log1p(df2[col])
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer(method='yeo-johnson')
df2[log_candidates] = pt.fit_transform(df2[log_candidates])

